<a href="https://colab.research.google.com/github/Alro10/PyTorch1.xTutorials/blob/master/04-Neural-Network/04_NeuralNets_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks

This is a tutorial for using shallow neural networks (NNets). The magic ReLU activation is a part of NNets arquitecture:

\begin{equation}
f(x)=x^{+}=\max (0, x)
\end{equation}

## Get dataset 

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration (use GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Feedforward Network

In [0]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        # Now it only takes a call to the layers to make predictions
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## Training step

In [3]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.2915
Epoch [1/5], Step [200/600], Loss: 0.3362
Epoch [1/5], Step [300/600], Loss: 0.1498
Epoch [1/5], Step [400/600], Loss: 0.1193
Epoch [1/5], Step [500/600], Loss: 0.0917
Epoch [1/5], Step [600/600], Loss: 0.1291
Epoch [2/5], Step [100/600], Loss: 0.0539
Epoch [2/5], Step [200/600], Loss: 0.1187
Epoch [2/5], Step [300/600], Loss: 0.2009
Epoch [2/5], Step [400/600], Loss: 0.0779
Epoch [2/5], Step [500/600], Loss: 0.0737
Epoch [2/5], Step [600/600], Loss: 0.0289
Epoch [3/5], Step [100/600], Loss: 0.0519
Epoch [3/5], Step [200/600], Loss: 0.1052
Epoch [3/5], Step [300/600], Loss: 0.0256
Epoch [3/5], Step [400/600], Loss: 0.0246
Epoch [3/5], Step [500/600], Loss: 0.0307
Epoch [3/5], Step [600/600], Loss: 0.0552
Epoch [4/5], Step [100/600], Loss: 0.0579
Epoch [4/5], Step [200/600], Loss: 0.0238
Epoch [4/5], Step [300/600], Loss: 0.0778
Epoch [4/5], Step [400/600], Loss: 0.0621
Epoch [4/5], Step [500/600], Loss: 0.0135
Epoch [4/5], Step [600/600], Loss:

## Accuracy 

In [4]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.73 %


## Save model

In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model_net.ckpt')

In [6]:
!ls

model_net.ckpt	sample_data
